### 0.00. Hello

In [14]:
import json

class FileItem:
    def __init__(self, fname):
        self._fname = fname

    # def __json_serialize__(self):
    #     return self.__dict__
    
f = FileItem("/foo/bar")
json.dumps(f.__dict__)  # OK
# json.dumps(f)  # TypeError: Object of type FileItem is not JSON serializable

'{"_fname": "/foo/bar"}'

In [9]:
import json

class Object:
    def toJSON(self):
        return json.dumps(self, default=lambda o: o.__dict__, sort_keys=True, indent=4)

me = Object()
me.name = "Onur"
me.age = 35
me.dog = Object()
me.dog.name = "Apollo"
# me.dog.age = 5

print(me.toJSON())

{
    "age": 35,
    "dog": {
        "name": "Apollo"
    },
    "name": "Onur"
}


In [10]:
from json import JSONEncoder

class MyEncoder(JSONEncoder):
    def default(self, o):
        return o.__dict__
    
MyEncoder().encode(f)

'{"_fname": "/foo/bar"}'

In [11]:
from json import JSONDecoder

def from_json(json_object):
    if '_fname' in json_object:
        return FileItem(json_object['_fname'])

f = JSONDecoder(object_hook=from_json).decode('{"_fname": "/foo/bar"}')
f

In [18]:
import jsonpickle
json_string = jsonpickle.encode(f)     # Seo : string < encoding < object
# json_string = jsonpickle.encode(f, indent=4)
re_f = jsonpickle.decode(json_string)  # Ods : object < decoding < string

print('1:', json_string)
print('2:', re_f)

1: {"py/object": "__main__.FileItem", "_fname": "/foo/bar"}
2: <__main__.FileItem object at 0x111f1df30>


In [39]:
import os
import jsonpickle
import time

file_name = '../data/00_1.FileItem.json'

class FileItem:
    def __init__(self, fname, age):
        self._fname = fname
        self._age = age

    def __repr__(self):
        return jsonpickle.encode(self.__dict__)

    def __str__(self):
        return jsonpickle.encode(self.__dict__)

obj1 = FileItem('/foo/bar', 12)
print('0:', vars(obj1))
print('-'*80)

with open(file_name, 'wt') as f:
    f.write(jsonpickle.encode(obj1))
time.sleep(1)
with open(file_name, 'rt') as f:
    obj2 = jsonpickle.decode(f.read())

print('1:', jsonpickle.encode(obj1, indent=4))
print('2:', jsonpickle.encode(obj2, indent=4))
print('-'*80)

print('3:', obj1)
print('3:', obj2.__dict__)
print('3:', obj2.__weakref__)
print('-'*80)

for key, value in FileItem.__dict__.items():
    print(f'4:      {key:<25} {value}')
print('-'*80)

# remove file
os.remove(file_name)

0: {'_fname': '/foo/bar', '_age': 12}
--------------------------------------------------------------------------------
1: {
    "py/object": "__main__.FileItem",
    "_fname": "/foo/bar",
    "_age": 12
}
2: {
    "py/object": "__main__.FileItem",
    "_fname": "/foo/bar",
    "_age": 12
}
--------------------------------------------------------------------------------
3: {"_fname": "/foo/bar", "_age": 12}
3: {'_fname': '/foo/bar', '_age': 12}
3: None
--------------------------------------------------------------------------------
4:      __module__                __main__
4:      __init__                  <function FileItem.__init__ at 0x1142f03a0>
4:      __repr__                  <function FileItem.__repr__ at 0x1142f12d0>
4:      __str__                   <function FileItem.__str__ at 0x1142f2e60>
4:      __dict__                  <attribute '__dict__' of 'FileItem' objects>
4:      __weakref__               <attribute '__weakref__' of 'FileItem' objects>
4:      __doc__           

In [44]:
class FileItem(dict):
    def __init__(self, fname):
        self._message = 'Hello, FileItem'
        dict.__init__(self, _fname=fname)

    def to_json(self):
        return self._message # or how you want it to be serialized

f = FileItem('tasks.txt')
print('1:', vars(f))
print('2:', json.dumps(f))  #No need to change anything here
print('3:', jsonpickle.encode(f))

1: {'_message': 'Hello, FileItem'}
2: {"_fname": "tasks.txt"}
3: {"py/object": "__main__.FileItem", "_fname": "tasks.txt", "__dict__": {"_message": "Hello, FileItem"}}


In [ ]:
# TODO: reading
from json import JSONEncoder

def _default(self, obj):
    return getattr(obj.__class__, "to_json", _default.default)(obj)

_default.default = JSONEncoder().default
JSONEncoder.default = _default


In [ ]:
# TODO: reading
import json
_fallback = json._default_encoder.default
json._default_encoder.default = lambda obj: getattr(obj.__class__,  "to_json", _fallback)(obj)

In [48]:
import json

class JsonSerializable(object):
    def toJson(self):
        return json.dumps(self.__dict__)

    def __repr__(self):
        return self.toJson()
    
    def __str__(self):
        return self.toJson()


class FileItem(JsonSerializable):
    def __init__(self, fname, age):
        self._fname = fname
        self._age = age

class FileItem2:
    def __init__(self, fname, age):
        self._fname = fname
        self._age = age

f = FileItem('/foo/bar', 12)
print('1:', f)
print('2:', str(f))
print('3:', f.toJson())

f2 = FileItem2('/foo/bar', 1234)
print('4:', json.dumps(f2, default=vars))


1: {"_fname": "/foo/bar", "_age": 12}
2: {"_fname": "/foo/bar", "_age": 12}
3: {"_fname": "/foo/bar", "_age": 12}
4: {"_fname": "/foo/bar", "_age": 1234}


In [46]:
import json
import inspect

class ObjectEncoder(json.JSONEncoder):
    def default(self, obj):
        if hasattr(obj, "to_json"):
            return self.default(obj.to_json())
        elif hasattr(obj, "__dict__"):
            d = dict(
                (key, value)
                for key, value in inspect.getmembers(obj)
                if not key.startswith("__")
                and not inspect.isabstract(value)
                and not inspect.isbuiltin(value)
                and not inspect.isfunction(value)
                and not inspect.isgenerator(value)
                and not inspect.isgeneratorfunction(value)
                and not inspect.ismethod(value)
                and not inspect.ismethoddescriptor(value)
                and not inspect.isroutine(value)
            )
            return self.default(d)
        return obj

class C(object):
    c = "NO"
    def to_json(self):
        return {"c": "YES"}

class B(object):
    b = "B"
    i = "I"
    def __init__(self, y):
        self.y = y
        
    def f(self):
        print("f")

class A(B):
    a = "A"
    def __init__(self):
        self.b = [{"ab": B("y")}]
        self.c = C()

print(json.dumps(A(), cls=ObjectEncoder, indent=2, sort_keys=True))


{
  "a": "A",
  "b": [
    {
      "ab": {
        "b": "B",
        "i": "I",
        "y": "y"
      }
    }
  ],
  "c": {
    "c": "YES"
  },
  "i": "I"
}


In [51]:
import json
from dataclasses import dataclass


@dataclass
class Company:
    id: int
    name: str

@dataclass
class User:
    id: int
    name: str
    email: str
    company: Company


company = Company(id=1, name="Example Ltd")
user = User(id=1, name="John Doe", email="john@doe.net", company=company)

print('1:', json.dumps(user, default=vars, indent=4))


{
    "id": 1,
    "name": "John Doe",
    "email": "john@doe.net",
    "company": {
        "id": 1,
        "name": "Example Ltd"
    }
}


In [8]:
import json
import jsonpickle

class User(object):
    def __init__(self, name, mail):
        self._name = name
        self._mail = mail

    def _asdict(self):
        return self.__dict__

print('1:', jsonpickle.encode(User('Alice', 'alice@mail.com'), indent=4))

def default(o):
    return o._asdict()

print('2:', json.dumps(User('Scott', 'scott@mail.com'), default=default))
print('3:', json.dumps(User('William', 'william@mail.com'), default=vars))
print('4:', json.dumps(User('Tiger', 'tiger@mail.com'), default=lambda __o: __o.__dict__))


1: {
    "py/object": "__main__.User",
    "_name": "Alice",
    "_mail": "alice@mail.com"
}
2: {"_name": "Scott", "_mail": "scott@mail.com"}
3: {"_name": "William", "_mail": "william@mail.com"}
4: {"_name": "Tiger", "_mail": "tiger@mail.com"}


In [9]:
import json
import jsonpickle

user = User('Scott', 'scott@mail.com')
print('1:', json.dumps(json.loads(jsonpickle.encode(user)), indent=2))
print('2:', user.__class__)
print('3:', user.__class__.__name__)

1: {
  "py/object": "__main__.User",
  "_name": "Scott",
  "_mail": "scott@mail.com"
}
2: <class '__main__.User'>
3: User


In [15]:
class Object(dict):
    def __init__(self, dict_=None):
        super().__init__(dict_ or {})

    def __getattr__(self, key):
        return self[key]

    def __setattr__(self, key, value):
        self[key] = value

# test
obj = Object()
obj.name = "John"
obj.age = 25
obj.brothers = [
    Object(),
    Object({'name': 'Jack', 'age': 23, 'address': 'NYC'}),
    Object({'name': 'Alice', 'age': 34, 'position': 'CEO'}),
]
text = json.dumps(obj, indent=4)
print('4:', text)

4: {
    "name": "John",
    "age": 25,
    "brothers": [
        {},
        {
            "name": "Jack",
            "age": 23,
            "address": "NYC"
        },
        {
            "name": "Alice",
            "age": 34,
            "position": "CEO"
        }
    ]
}


In [19]:
class FileItem(dict):
    def __init__(self, fname):
        # self['fname'] = fname
        # fname = fname
        pass

    #fname property
    fname: str = property()

    @fname.getter
    def fname(self):
        # return getattr(self.__class__,  'fname', None)
        return self['fname']

    @fname.setter
    def fname(self, value: str):
        # setattr(self.__class__, 'fname', value)
        self['fname'] = value

    #Repeat for other properties

f = FileItem('/foo/bar')
f.fname('/hello/world')
print('5:', f)

KeyError: 'fname'

In [21]:
import json, sys, os

class File:
    def __init__(self, path):
        self.path = path

    def isdir(self):
        return os.path.isdir(self.path)

    def isfile(self):
        return os.path.isfile(self.path)

    def children(self):        
        return [File(os.path.join(self.path, f)) for f in os.listdir(self.path)]

    def getsize(self):        
        return os.path.getsize(self.path)

    def getModificationTime(self):
        return os.path.getmtime(self.path)

def _default(o):
    d = {}
    d['path'] = o.path
    d['isFile'] = o.isfile()
    d['isDir'] = o.isdir()
    d['mtime'] = int(o.getModificationTime())
    d['size'] = o.getsize() if o.isfile() else 0
    if o.isdir(): d['children'] = o.children()
    return d

folder = os.path.abspath('.')
json.dump(File(folder), sys.stdout, default=_default, indent=4)


{
    "path": "/Users/kang-air/KANG/_GIT_HUB/python/Python-Cookbook-3rd-edition-JupyterNotebook-Code/Chapter 00",
    "isFile": false,
    "isDir": true,
    "mtime": 1711197986,
    "size": 0,
    "children": [
        {
            "path": "/Users/kang-air/KANG/_GIT_HUB/python/Python-Cookbook-3rd-edition-JupyterNotebook-Code/Chapter 00/0.00. Hello.ipynb",
            "isFile": true,
            "isDir": false,
            "mtime": 1711209731,
            "size": 16869
        }
    ]
}

In [ ]:
import json

class Serializer(object):
    ''' Serializer.serialize(object) '''
    @staticmethod
    def serialize(object):
        return json.dumps(object, default=lambda o: o.__dict__.values()[0])


In [25]:
# Your custom class
class MyCustom(object):
    def __json__(self):
        return {
            'a': self.a,
            'b': self.b,
            '__python__': 'mymodule.submodule:MyCustom.from_json',
        }

    to_json = __json__  # supported by simplejson

    @classmethod
    def from_json(cls, json):
        obj = cls()
        obj.a = json['a']
        obj.b = json['b']
        return obj

# Dumping and loading
import jsonpickle

obj = MyCustom()
obj.a = 3
obj.b = 4

json = jsonpickle.encode(obj)
print('1:', json)

# Two-step loading
obj2_dict = jsonpickle.decode(json)
obj2 = MyCustom.from_json(MyCustom, obj2_dict)
print('2:', jsonpickle.encode(obj2))

# Make sure we have the correct thing
# assert isinstance(obj2, MyCustom)
# assert obj2.__dict__ == obj.__dict__


1: {"py/object": "__main__.MyCustom", "a": 3, "b": 4}


TypeError: MyCustom.from_json() takes 2 positional arguments but 3 were given

In [28]:
class JsonSerializable(object):

    def serialize(self):
        return json.dumps(self.__dict__)

    def __repr__(self):
        return self.serialize()

    @staticmethod
    def dumper(obj):
        if "serialize" in dir(obj):
            return obj.serialize()
        return obj.__dict__

class FileItem(JsonSerializable):
    def __init__(self, fname, age):
        self._fname = fname
        self._age = age

f = FileItem('/foo/bar', 12)
f.serialize()

AttributeError: 'str' object has no attribute 'dumps'

In [29]:
import json

class Foo(object):
    def __init__(self):
        self.bar = 'baz'
        self._qux = 'flub'

    def somemethod(self):
        pass

'''
The parameter default(obj) is a function that should return a 
serializable version of obj or raise TypeError. The default 
default simply raises TypeError. 

https://docs.python.org/3.4/library/json.html#json.dumps
'''
def default(instance):
    return {k: v
            for k, v in vars(instance).items()
            if not str(k).startswith('_')}

json_foo = json.dumps(Foo(), default=default)
assert '{"bar": "baz"}' == json_foo

print(json_foo)


{"bar": "baz"}


In [ ]:
class CustomEncodeCls:
    def __init__(self):
        self.relevant = 42
        self.irrelevant = 37

    def __json_encode__(self):
        # should return primitive, serializable types like dict, list, int, string, float...
        return {'relevant': self.relevant}

    def __json_decode__(self, **attrs):
        # should initialize all properties; note that __init__ is not called implicitly
        self.relevant = attrs['relevant']
        self.irrelevant = 12


In [ ]:
'''
Created on 2020-09-03

@author: wf
'''
import json

class JSONAble(object):
    '''
    mixin to allow classes to be JSON serializable see
    https://stackoverflow.com/questions/3768895/how-to-make-a-class-json-serializable
    '''

    def __init__(self):
        '''
        Constructor
        '''
    
    def toJSON(self):
        return json.dumps(self, default=lambda o: o.__dict__, 
            sort_keys=True, indent=4)
        
    def getValue(self,v):
        if (hasattr(v, "asJSON")):
            return v.asJSON()
        elif type(v) is dict:
            return self.reprDict(v)
        elif type(v) is list:
            vlist=[]
            for vitem in v:
                vlist.append(self.getValue(vitem))
            return vlist
        else:   
            return v
    
    def reprDict(self,srcDict):
        '''
        get my dict elements
        '''
        d = dict()
        for a, v in srcDict.items():
            d[a]=self.getValue(v)
        return d
    
    def asJSON(self):
        '''
        recursively return my dict elements
        '''
        return self.reprDict(self.__dict__)

In [ ]:
def testJsonAble(self):
    family=Family("The Flintstones")
    family.add(Person("Fred","Flintstone")) 
    family.add(Person("Wilma","Flintstone"))
    json1=family.toJSON()
    json2=family.asJSON()
    print(json1)
    print(json2)

class Family(JSONAble):
    def __init__(self,name):
        self.name=name
        self.members={}
    
    def add(self,person):
        self.members[person.lastName+","+person.firstName]=person

class Person(JSONAble):
    def __init__(self,firstName,lastName):
        self.firstName=firstName
        self.lastName=lastName


In [30]:
#!/usr/bin/env python3

import json
from dataclasses import dataclass

@dataclass
class Person:
    first: str
    last: str
    age: int

    @property
    def __json__(self):
        return {
            "name": f"{self.first} {self.last}",
            "age": self.age
        }

john = Person("John", "Doe", 42)
print('1:', json.dumps(john, indent=4, default=lambda x: x.__json__))


1: {
    "name": "John Doe",
    "age": 42
}


In [33]:
import jsonpickle

class DataModel:
    def __init__(self, id, value):
        self._id = id
        self._value = value

    def __json__(self):
        return jsonpickle.encode(self.__dict__)

print('1:', DataModel(1, 'foo').__json__())
print('2:', jsonpickle.encode(DataModel(2, 'bar')))

1: {"_id": 1, "_value": "foo"}
2: {"py/object": "__main__.DataModel", "_id": 2, "_value": "bar"}


In [ ]:
def getSerializable(doc):
    # check if it's a list
    if isinstance(doc, list):
        for i, val in enumerate(doc):
            doc[i] = getSerializable(doc[i])
        return doc

    # check if it's a dict
    if isinstance(doc, dict):
        for key in doc.keys():
            doc[key] = getSerializable(doc[key])
        return doc

    # Process ObjectId
    # if isinstance(doc, ObjectId):
    if isinstance(doc, str):
        doc = str(doc)
        return doc

    # Use any other custom serializting stuff here...

    # For the rest of stuff
    return doc



In [ ]:
class DObject(json.JSONEncoder):
    def delete_not_related_keys(self, _dict):
        for key in ["skipkeys", "ensure_ascii", "check_circular", "allow_nan", "sort_keys", "indent"]:
            try:
                del _dict[key]
            except:
                continue

    def default(self, o):
        if hasattr(o, '__dict__'):
            my_dict = o.__dict__.copy()
            self.delete_not_related_keys(my_dict)
            return my_dict
        else:
            return o

a = DObject()
a.name = 'abdul wahid'
b = DObject()
b.name = a

print(json.dumps(b, cls=DObject))


In [ ]:
def sterilize(obj):
    """Make an object more ameniable to dumping as json
    """
    if type(obj) in (str, float, int, bool, type(None)):
        return obj
    elif isinstance(obj, dict):
        return {k: sterilize(v) for k, v in obj.items()}
    list_ret = []
    dict_ret = {}
    for a in dir(obj):
        if a == '__iter__' and callable(obj.__iter__):
            list_ret.extend([sterilize(v) for v in obj])
        elif a == '__dict__':
            dict_ret.update({k: sterilize(v) for k, v in obj.__dict__.items() if k not in ['__module__', '__dict__', '__weakref__', '__doc__']})
        elif a not in ['__doc__', '__module__']:
            aval = getattr(obj, a)
            if type(aval) in (str, float, int, bool, type(None)):
                dict_ret[a] = aval
            elif a != '__class__' and a != '__objclass__' and isinstance(aval, type):
                dict_ret[a] = sterilize(aval)
    if len(list_ret) == 0:
        if len(dict_ret) == 0:
            return repr(obj)
        return dict_ret
    else:
        if len(dict_ret) == 0:
            return list_ret
    return (list_ret, dict_ret)


In [ ]:
class Serializer(object):
    @staticmethod
    def serialize(obj):
        def check(o):
            for k, v in o.__dict__.items():
                try:
                    _ = json.dumps(v)
                    o.__dict__[k] = v
                except TypeError:
                    o.__dict__[k] = str(v)
            return o
        return json.dumps(check(obj).__dict__, indent=2)


In [34]:
print('Hello, world!!')

Hello, world!!
